In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor


In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:25688
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (29.193s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 25688
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 3241 (parent (waf shell) id: 3105)
Waiting for Python process to connect on port: tcp://localhost:25688
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(state_size=50, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/cd156acce1954a69b9c129a33373f91b

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 333/6300 [02:38<56:25,  1.76it/s, curr_speed=31.80 Mbps, mb_sent=10.18 Mb]

------- STARTED TRAINING -------


...

Waiting for simulation script to connect on port: tcp://localhost:42707
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.010s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 42707
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 935374 (parent (waf shell) id: 935347)
Waiting for Python process to connect on port: tcp://localhost:42707
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/cd156acce1954a69b9c129a33373f91b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2545.439453125, 3090.641845703125)
COMET INFO:     Current throughput [89985]  : (16.754148483276367, 58.829402923583984)
COMET INFO:     Fairness index [89985]      : (0.025684000924229622, 0.9953569769859314)
COMET INFO:     Megabytes sent [89985]      : (0.17871099710464478, 2234.523248672485

Sent 2234.52 Mb/s.	Mean speed: 37.24 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.


# Execution num2:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:38320
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (26.497s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 38320
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 3279 (parent (waf shell) id: 3169)
Waiting for Python process to connect on port: tcp://localhost:38320
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(history_length, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/24ce603cc4d84089b7730023af228fd0

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 333/6300 [01:14<25:36,  3.88it/s, curr_speed=31.27 Mbps, mb_sent=10.01 Mb]

------- STARTED TRAINING -------


...

Sent 2221.10 Mb/s.	Mean speed: 37.02 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [58:09<00:00,  1.81it/s, curr_speed=36.16 Mbps, mb_sent=2224.50 Mb]


Waiting for simulation script to connect on port: tcp://localhost:17785
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.024s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 17785
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 1066317 (parent (waf shell) id: 1066298)
Waiting for Python process to connect on port: tcp://localhost:17785
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/24ce603cc4d84089b7730023af228fd0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2514.716064453125, 3090.26953125)
COMET INFO:     Current throughput [89985]  : (18.739822387695312, 59.14214324951172)
COMET INFO:     Fairness index [89985]      : (0.02563999965786934, 0.9955750107765198)
COMET INFO:     Megabytes sent [89985]      : (0.18987999856472015, 2224.504322528839)

Sent 2224.50 Mb/s.	Mean speed: 37.08 Mb/s	Episode 15/15 finished

Training finished.


# Execution num3:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:31831
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (26.079s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 31831
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 3306 (parent (waf shell) id: 3285)
Waiting for Python process to connect on port: tcp://localhost:31831
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(history_length, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/3a9b8664399f40c3b2bb0d5abbc7412d

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 334/6300 [01:21<32:14,  3.08it/s, curr_speed=33.61 Mbps, mb_sent=11.09 Mb]

------- STARTED TRAINING -------


...

Sent 2253.75 Mb/s.	Mean speed: 37.56 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [1:29:44<00:00,  1.17it/s, curr_speed=36.48 Mbps, mb_sent=2250.74 Mb]


Waiting for simulation script to connect on port: tcp://localhost:31179
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.369s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 31179
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 1201255 (parent (waf shell) id: 1201230)
Waiting for Python process to connect on port: tcp://localhost:31179
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/3a9b8664399f40c3b2bb0d5abbc7412d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2539.944091796875, 3140.93408203125)
COMET INFO:     Current throughput [89985]  : (20.104999542236328, 48.41948699951172)
COMET INFO:     Fairness index [89985]      : (0.025679999962449074, 0.9953939914703369)
COMET INFO:     Megabytes sent [89985]      : (0.22338899970054626, 2253.745910674

Sent 2250.74 Mb/s.	Mean speed: 37.51 Mb/s	Episode 15/15 finished

Training finished.


# Execution num4:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent, Config
from agents.ddpg.model import Actor
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:26077
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.268s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 26077
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 1240906 (parent (waf shell) id: 1240885)
Waiting for Python process to connect on port: tcp://localhost:26077
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr_actor = 4e-4
lr_critic = 4e-3

config = Config(buffer_size=4*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr_actor=lr_actor, lr_critic=lr_critic, update_every=1)
agent = Agent(history_length, action_size=1, config=config, actor_layers=[8, 128, 16], critic_layers=[8,128,16])

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"Actor: {lr_actor}",
        f"Critic: {lr_critic}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
# logger = teacher.eval(agent,
#                         simTime=simTime,
#                         stepTime=stepTime,
#                         history_length=history_length,
#                         tags=tags,
#                         parameters=hyperparams)
# agent.save()

CuDNN version: 7102
cpu


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/a2e96f4b69904af8be2272f4969ff4d6

  0%|          | 0/6300 [00:00<?, ?it/s]

0


  5%|▌         | 333/6300 [02:01<44:38,  2.23it/s, curr_speed=30.30 Mbps, mb_sent=9.70 Mb]

------- STARTED TRAINING -------


...

Sent 2231.68 Mb/s.	Mean speed: 37.19 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [57:09<00:00,  1.84it/s, curr_speed=37.75 Mbps, mb_sent=2226.70 Mb]


Waiting for simulation script to connect on port: tcp://localhost:38684
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (1.922s)


Ns3Env parameters:
--nWifi: 50
--simulationTime: 60
--openGymPort: 38684
--envStepTime: 0.01
--seed: -1
--agentType: continuous
--scenario: convergence
--dryRun: 0
Simulation started
Simulation process id: 2028360 (parent (waf shell) id: 2028347)
Waiting for Python process to connect on port: tcp://localhost:38684
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/a2e96f4b69904af8be2272f4969ff4d6
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2505.720947265625, 3101.870849609375)
COMET INFO:     Current throughput [89985]  : (15.637200355529785, 69.94298553466797)
COMET INFO:     Fairness index [89985]      : (0.02566000074148178, 0.9955629706382751)
COMET INFO:     Megabytes sent [89985]      : (0.15637199580669403, 2234.836013808

Sent 2226.70 Mb/s.	Mean speed: 37.11 Mb/s	Episode 15/15 finished

Training finished.
